# Housing, Claims, and Return Periods for Zips
_Calvin Whealton_

This notebooks makes plots of the Zillow housing value index (ZHVI), flood insurance claims, and return periods of floods. Only the available data for each zip code is plotted.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import numpy as np

Reading in the relevant files. Each of the Zillow (housing price), National Flood Insurance Program claims, and stream gages will be read-in separately.

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
housing = pd.read_csv('zillow_mon_pct_val.csv')

In [ ]:
housing.head()

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
claims = pd.read_csv('ts_claims_month.csv')

In [ ]:
claims.head()

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
ret_pers = pd.read_csv('ts_rps_2020-08-15.csv')

In [ ]:
ret_pers.head()

In [ ]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')

zip_gage = pd.read_csv('zip_gage_dist_2020-08-10.csv')

In [ ]:
zip_gage.head()

In [ ]:
zip_gage.shape[0]

In [ ]:
claims.shape[0]

In [ ]:
housing.shape[0]

# making figures

In [ ]:
# loop through all the zip codes that are in the
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/geo_data/tl_2019_us_zcta510_clipped48contig')
zip_shape = gpd.read_file('clipped48contig.shp')
zip_shape['GEOID10_str'] = zip_shape['GEOID10'].apply(lambda x: '{0:0>5}'.format(x))


In [ ]:
size=15
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,8),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 0,
          'axes.titlepad': 25}

plt.rcParams.update(params)

x_names = (housing.columns[2:housing.shape[1]])[0:288]

for z in zip_shape['GEOID10'].astype(int).values[29071:zip_shape.shape[0]]:
    if z in housing['GEOID10_str'].astype(int).values:
        os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_housing_vals')
        hou_temp = housing.loc[housing['GEOID10_str'].astype(int) == z]
        
        plt.figure(figsize=(10,5))
        x_vals = np.arange(0,len(x_names))
        y_vals = hou_temp.iloc[0][x_names]
        plt.plot(x_vals,y_vals,color='#bcbddc')
        plt.xlabel('Date')
        plt.ylabel('% Change (M-over-M)')
        plt.xticks(np.arange(-1,len(x_vals),12), np.arange(1996,2021), rotation='vertical')
        plt.title('Zillow Housing Value Index')
        plt.savefig(str(z).zfill(5) + '_housing'+'.png')
        plt.close()
        

    if z in claims['GEOID10_str'].astype(int).values:
        os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_claims')
        cla_temp = claims.loc[claims['GEOID10_str'].astype(int) == z]
        
        plt.figure(figsize=(10,5))
        claims_temp = claims.loc[claims['GEOID10_str'].astype(int) == z]
        x_vals = np.arange(0,len(x_names))
        y_vals = np.append(np.array(claims_temp.iloc[0][-284:]),np.zeros(4))
        if max(y_vals) != 0:
            plt.bar(x_vals,y_vals,color='#4daf4a')
            plt.xlabel('Date')
            plt.ylabel('Claims (USD)')
            plt.xticks(np.arange(-1,len(x_vals),12), np.arange(1996,2021), rotation='vertical')
            plt.title('National Flood Insurance Claims')
        plt.savefig(str(z).zfill(5) + '_zip_claims'+'.png')
        plt.close()
            
    if z in zip_gage['GEOID10'].astype(int).values:
        os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/visualizations/zip_results/zip_rps')
        
        gage_lookup_temp = zip_gage.loc[zip_gage['GEOID10'] == z]
        plt.figure(figsize=(10,5))
        gage_cols = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a']
        for j in range(10):
            if  gage_lookup_temp['gage'+str(j)].values[0] in ret_pers['Gage'].values:
                plt.scatter(np.arange(1996,2020),\
                            np.ndarray.flatten((ret_pers.loc[ret_pers['Gage'].values==gage_lookup_temp['gage'+str(j)].values[0]]).values)[-24:],\
                            c=gage_cols[j])

        plt.yscale('log')
        plt.xlabel('Date')
        plt.xticks(np.arange(1996,2020), np.arange(1996,202), rotation='vertical')
        plt.ylabel('Return Period (yrs)')
        plt.title('Extremes in Flood Series')
        plt.ylim((1,500))
        plt.savefig(str(z).zfill(5) + '_zip_rps'+'.png')
        plt.close()

In [ ]:
gage_lookup_temp['gage'+str(j)].values[0] in ret_pers['Gage'].values

In [ ]:
np.arange(1996,2020)